<a href="https://colab.research.google.com/github/crystalloide/RAG/blob/main/LAB31_Planning_vs_Reactive_Agents_avec_LangChain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LAB31 : Planning vs Reactive Loops avec LangChain

**Objectif:** Construire deux agents LangChain simples:
- **Reactive Agent** → répond directement, outil par outil
- **Planning Agent** → décompose en sous-tâches, puis exécute

**Durée estimée:** 20-30 minutes

**Livrable:** Notebook comparant les sorties des deux patterns sur les mêmes requêtes

## Step 1: Setup

Installation des dépendances nécessaires

In [1]:
# Installation des packages
!pip install -q langchain langchain-openai openai python-dotenv
!pip install -q langgraph langchain-core

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.8/84.8 kB 1.9 MB/s eta 0:00:00


In [2]:
# Configuration de la clé API OpenAI
# Dans Google Colab, nous utilisons les secrets au lieu de .env
import os
from google.colab import userdata

# Option 1: Utiliser les secrets Colab (recommandé)
# Allez dans le menu à gauche (icône clé) > Secrets > Ajouter "OPENAI_API_KEY"
try:
    os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
    print("✅ Clé API chargée depuis les secrets Colab")
except:
    print("⚠️ Secret non trouvé. Utilisation de l'input manuel...")
    # Option 2: Input manuel (moins sécurisé)
    import getpass
    os.environ['OPENAI_API_KEY'] = getpass.getpass('Entrez votre clé API OpenAI: ')

✅ Clé API chargée depuis les secrets Colab


## Step 2: Définition des Tools

Création de deux outils simples:
1. **Calculator** - Effectue des calculs mathématiques
2. **Knowledge Base Lookup** - Recherche dans une base de connaissances

In [3]:
from langchain_core.tools import Tool
from langchain_openai import ChatOpenAI

# Outil 1: Calculatrice
def calculator(expr: str) -> str:
    """
    Évalue une expression mathématique.
    Exemple: "15*4" -> "60"
    """
    try:
        result = eval(expr)
        return str(result)
    except Exception as e:
        return f"Erreur: {e}"

# Outil 2: Base de connaissances
kb = {
    "LangChain": "Un framework pour construire des agents alimentés par des LLM.",
    "RAG": "Retrieval-Augmented Generation, combinant récupération et génération.",
    "CrewAI": "Un framework pour l'orchestration multi-agents."
}

def lookup(term: str) -> str:
    """
    Recherche un terme dans la base de connaissances.
    """
    return kb.get(term, f"Aucune entrée pour '{term}'")

# Définition des outils pour LangChain
tools = [
    Tool(
        name="Calculator",
        func=calculator,
        description="Utilisé pour effectuer des calculs mathématiques. Input: une expression mathématique comme '15*4' ou '25*3+10'."
    ),
    Tool(
        name="Lookup",
        func=lookup,
        description="Utilisé pour rechercher des informations dans la base de connaissances. Input: un terme comme 'LangChain', 'RAG', ou 'CrewAI'."
    ),
]

print("✅ Outils définis:")
for tool in tools:
    print(f"  - {tool.name}: {tool.description}")

✅ Outils définis:
  - Calculator: Utilisé pour effectuer des calculs mathématiques. Input: une expression mathématique comme '15*4' ou '25*3+10'.
  - Lookup: Utilisé pour rechercher des informations dans la base de connaissances. Input: un terme comme 'LangChain', 'RAG', ou 'CrewAI'.


### Test des outils

In [4]:
# Test de la calculatrice
print("Test Calculator:")
print(f"  15*4 = {calculator('15*4')}")
print(f"  25*3+10 = {calculator('25*3+10')}")
print(f"  12/4 = {calculator('12/4')}")

print("\nTest Lookup:")
print(f"  LangChain: {lookup('LangChain')}")
print(f"  RAG: {lookup('RAG')}")
print(f"  CrewAI: {lookup('CrewAI')}")
print(f"  Unknown: {lookup('Unknown')}")

Test Calculator:
  15*4 = 60
  25*3+10 = 85
  12/4 = 3.0

Test Lookup:
  LangChain: Un framework pour construire des agents alimentés par des LLM.
  RAG: Retrieval-Augmented Generation, combinant récupération et génération.
  CrewAI: Un framework pour l'orchestration multi-agents.
  Unknown: Aucune entrée pour 'Unknown'


## Step 3: Reactive Agent (20 min)

Un agent réactif simple: le LLM décide quel outil utiliser → exécute → répond.

Le pattern **ReAct** (Reasoning + Acting) est utilisé:
1. **Thought**: L'agent réfléchit à la prochaine action
2. **Action**: Il exécute un outil
3. **Observation**: Il observe le résultat
4. Répète jusqu'à avoir la réponse finale

In [10]:
from langchain.agents import create_agent
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import tool

### Agent Réactif : exemple n°1

In [11]:
# Initialize LLM
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Define your tools (example)
from langchain_core.tools import tool

@tool
def calculator(expression: str) -> str:
    """Useful for mathematical calculations. Input should be a valid Python expression like '15*4'"""
    try:
        return str(eval(expression))
    except:
        return "Invalid expression"

tools = [calculator]

# Create the agent (LangGraph's create_react_agent doesn't need a prompt template)
agent_executor = create_agent(llm, tools)

# Run the agent
print("\n--- Reactive Agent ---")
result = agent_executor.invoke({
    "messages": [("user", "What is 15*4 and also explain what LangChain is?")]
})

# Print the final response
print(result["messages"][-1].content)


--- Reactive Agent ---
The result of \( 15 \times 4 \) is \( 60 \).

### What is LangChain?

LangChain is a framework designed for developing applications that utilize large language models (LLMs). It provides a structured way to build applications that can interact with LLMs, enabling developers to create more complex and capable systems. Key features of LangChain include:

1. **Modularity**: LangChain allows developers to compose different components, such as prompt templates, memory, and chains, to create sophisticated workflows.

2. **Integration**: It supports integration with various data sources and APIs, making it easier to pull in external information and enhance the capabilities of LLMs.

3. **Chains**: LangChain enables the creation of chains, which are sequences of calls to LLMs or other functions, allowing for more complex interactions and processing.

4. **Memory**: It provides mechanisms for memory, allowing applications to remember past interactions and maintain contex

### Agent Réactif : exemple n°2

In [15]:
# Les Tools :
def calculator(expr: str) -> str:
    """Évalue une expression mathématique."""
    try:
        result = eval(expr)
        return str(result)
    except Exception as e:
        return f"Erreur: {e}"

kb = {
    "LangChain": "Un framework pour construire des agents alimentés par des LLM.",
    "RAG": "Retrieval-Augmented Generation, combinant récupération et génération.",
    "CrewAI": "Un framework pour l'orchestration multi-agents."
}

def lookup(term: str) -> str:
    """Recherche un terme dans la base de connaissances."""
    return kb.get(term, f"Aucune entrée pour '{term}'")

tools = [
    Tool(
        name="Calculator",
        func=calculator,
        description="Utilisé pour effectuer des calculs mathématiques. Input: une expression mathématique comme '15*4' ou '25*3+10'."
    ),
    Tool(
        name="Lookup",
        func=lookup,
        description="Utilisé pour rechercher des informations dans la base de connaissances. Input: un terme comme 'LangChain', 'RAG', ou 'CrewAI'."
    ),
]

# Initialisation du LLM
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Création de l'agent avec LangGraph
reactive_agent = create_agent(llm, tools)

print("✅ Agent réactif créé avec succès")


✅ Agent réactif créé avec succès


### Test du Reactive Agent

In [16]:
print("\n" + "="*80)
print("REACTIVE AGENT - Query 1")
print("="*80)

query1 = "Qu'est-ce que 15*4 et explique aussi ce qu'est LangChain?"
# result1 = reactive_agent.invoke(query1)
result1 = reactive_agent.invoke({
    "messages": [("user", query1)]
})

print("\n📝 Réponse:\n\n", result1["messages"][-1].content)

# Autre Utilisation de l'agent
print("\n" + "="*80)
print("REACTIVE AGENT - Query 2")
print("="*80)
response = reactive_agent.invoke({
    "messages": [("user", "Que représente MongoDB et calcule 150*400*12*24")]
})

print("\n📝 Réponse:", response["messages"][-1].content)



REACTIVE AGENT - Query 1

📝 Réponse:

 Le résultat de \( 15 \times 4 \) est 60.

LangChain est un framework conçu pour construire des agents alimentés par des modèles de langage (LLM).

REACTIVE AGENT - Query 2

📝 Réponse: MongoDB est une base de données NoSQL orientée documents, qui permet de stocker des données sous forme de documents JSON. Elle est conçue pour être évolutive et flexible, facilitant le développement d'applications modernes.

En ce qui concerne le calcul, \(150 \times 400 \times 12 \times 24\) donne \(17,280,000\).


**👉 Observation:** L'agent réactif interagit pas à pas:
- Il analyse la question
- Décide d'utiliser un et quel outil
- Observe le résultat
- Décide de la prochaine action
- Continue jusqu'à avoir une réponse complète

## Step 4: Planning Agent :

Un agent planificateur décompose d'abord la requête en sous-tâches, puis les exécute.

**Planning = décomposer la requête en sous-tâches d'abord, puis exécuter**

In [17]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [18]:
# Template de prompt pour la planification
plan_prompt = PromptTemplate.from_template("""
Tu es un agent de planification. Décompose la requête suivante en sous-tâches avant de répondre:
Requête: {task}
Retourne le plan sous la forme:
1. Sous-tâche A
2. Sous-tâche B
...
Sois précis et indique clairement les outils nécessaires (Calculator ou Lookup).
""")

# Chaîne de planification (syntaxe LCEL)
plan_chain = plan_prompt | llm | StrOutputParser()

def planning_agent(task: str) -> str:
    """
    Agent planificateur: crée un plan puis l'exécute.
    """
    # Étape 1: Créer le plan
    print("\n🧠 PHASE DE PLANIFICATION...")
    plan = plan_chain.invoke({"task": task})
    print("\n📋 PLAN GÉNÉRÉ:")
    print("-" * 60)
    print(plan)
    print("-" * 60)

    # Étape 2: Exécution simple (détection par mots-clés)
    print("\n⚙️ PHASE D'EXÉCUTION...")
    answer_parts = []

    # Détection de calculs
    import re
    calc_patterns = re.findall(r'\d+[\*\/\+\-]\d+', task)
    if calc_patterns or "calcul" in task.lower() or "*" in task:
        # Extraire l'expression mathématique
        for expr in calc_patterns:
            result = calculator(expr)
            answer_parts.append(f"📊 Calcul de {expr}: {result}")

    # Détection de termes à rechercher
    for term in kb.keys():
        if term in task:
            result = lookup(term)
            answer_parts.append(f"📚 Information sur {term}: {result}")

    print("\n✅ RÉSULTATS D'EXÉCUTION:")
    for part in answer_parts:
        print(f"  {part}")

    return "\n".join(answer_parts)
    print("✅ Agent planificateur créé avec succès")

### Test du Planning Agent

In [19]:
print("\n" + "="*80)
print("PLANNING AGENT - Query 1")
print("="*80)

query1 = "Qu'est-ce que 15*4 et explique aussi ce qu'est LangChain?"
result1 = planning_agent(query1)

print("\n" + "="*80)
print("RÉPONSE FINALE:")
print("="*80)
print(result1)


PLANNING AGENT - Query 1

🧠 PHASE DE PLANIFICATION...

📋 PLAN GÉNÉRÉ:
------------------------------------------------------------
Voici le plan décomposé en sous-tâches :

1. Sous-tâche A : Calculer le produit de 15 et 4.
   - Outil nécessaire : Calculator

2. Sous-tâche B : Rechercher des informations sur LangChain.
   - Outil nécessaire : Lookup

Une fois ces sous-tâches réalisées, je pourrai fournir la réponse complète à la requête.
------------------------------------------------------------

⚙️ PHASE D'EXÉCUTION...

✅ RÉSULTATS D'EXÉCUTION:
  📊 Calcul de 15*4: 60
  📚 Information sur LangChain: Un framework pour construire des agents alimentés par des LLM.

RÉPONSE FINALE:
📊 Calcul de 15*4: 60
📚 Information sur LangChain: Un framework pour construire des agents alimentés par des LLM.


**👉 Observation:** L'agent planificateur:
1. **D'abord** crée un plan structuré
2. **Ensuite** exécute les sous-tâches en séquence
3. Structure globale plus claire

## Step 5: Comparaison des méthodes  :

Comparaison directe des deux agents sur les mêmes requêtes

In [20]:
# Requêtes de test
test_queries = [
    "Calcule 25*3+10?",
    "Dis-moi ce qu'est CrewAI, puis calcule 12/4.",
    "Calcule 100*2 et explique RAG."
]

print("\n" + "#"*80)
print("# COMPARAISON DES DEUX AGENTS")
print("#"*80)

for i, query in enumerate(test_queries, 1):
    print(f"\n\n{'='*80}")
    print(f"QUERY {i}: {query}")
    print("="*80)

    # Reactive Agent
    print("\n" + "▶"*40)
    print("REACTIVE AGENT")
    print("▶"*40)
    try:
        reactive_result= reactive_agent.invoke({
         "messages": [("user", query)]   })
        print(f"\n✅ Résultat: {reactive_result["messages"][-1].content}")
    except Exception as e:
        print(f"\n❌ Erreur: {e}")



    # Planning Agent
    print("\n" + "▶"*40)
    print("PLANNING AGENT")
    print("▶"*40)
    try:
        planning_result = planning_agent(query)
        print(f"\n✅ Résultat: {planning_result}")
    except Exception as e:
        print(f"\n❌ Erreur: {e}")


################################################################################
# COMPARAISON DES DEUX AGENTS
################################################################################


QUERY 1: Calcule 25*3+10?

▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶
REACTIVE AGENT
▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶

✅ Résultat: Le résultat de \( 25 \times 3 + 10 \) est 85.

▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶
PLANNING AGENT
▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶

🧠 PHASE DE PLANIFICATION...

📋 PLAN GÉNÉRÉ:
------------------------------------------------------------
Pour répondre à la requête "Calcule 25*3+10", voici le plan décomposé en sous-tâches :

1. Sous-tâche A : Calculer le produit de 25 et 3.
   - Outil nécessaire : Calculator

2. Sous-tâche B : Ajouter 10 au résultat obtenu de la sous-tâche A.
   - Outil nécessaire : Calculator

3. Sous-tâche C : Fournir le résultat final de l'opération.
   - Outil nécessaire : Aucun (juste la présentation du résultat)

Ainsi, les so

### Observations sur les différences

**Reactive Agent:**
- ✅ Répond rapidement et directement
- ✅ Adapte dynamiquement sa stratégie
- ⚠️ Peut perdre la structure globale
- ⚠️ Raisonnement moins visible

**Planning Agent:**
- ✅ Structure claire et explicite
- ✅ Décomposition visible des tâches
- ✅ Meilleure fiabilité sur tâches complexes
- ⚠️ Prend plus de temps (2 appels LLM)
- ⚠️ Moins flexible si le plan initial est incorrect

## Step 6: Mini-projet - Enhanced Planning Agent :

Amélioration de l'agent planificateur avec:
1. Parsing des sous-tâches avec regex
2. Ajout d'outils supplémentaires
3. Exécution séquentielle avec résultats inter-étapes

### Ajout de nouveaux outils

In [21]:
# Nouvel outil: Résumeur de texte
def summarizer(text: str) -> str:
    """
    Résume un texte (simulation simple).
    """
    words = text.split()
    if len(words) <= 10:
        return text
    return " ".join(words[:10]) + "..."

# Nouvel outil: Recherche web (simulée)
def web_search(query: str) -> str:
    """
    Simule une recherche web.
    """
    mock_results = {
        "python": "Python est un langage de programmation interprété, de haut niveau.",
        "ai": "L'Intelligence Artificielle (IA) est la simulation de processus d'intelligence humaine.",
        "langchain": "LangChain est un framework pour développer des applications alimentées par des LLM."
    }

    for key, value in mock_results.items():
        if key in query.lower():
            return value
    return f"Aucun résultat trouvé pour '{query}'"

# Outils enrichis
enhanced_tools = [
    Tool(
        name="Calculator",
        func=calculator,
        description="Calcule des expressions mathématiques."
    ),
    Tool(
        name="Lookup",
        func=lookup,
        description="Recherche dans la base de connaissances."
    ),
    Tool(
        name="Summarizer",
        func=summarizer,
        description="Résume un texte long."
    ),
    Tool(
        name="WebSearch",
        func=web_search,
        description="Recherche des informations sur le web (simulé)."
    ),
]

print("✅ Nouveaux outils ajoutés:")
for tool in enhanced_tools:
    print(f"  - {tool.name}")

✅ Nouveaux outils ajoutés:
  - Calculator
  - Lookup
  - Summarizer
  - WebSearch


## Comparaison finale des 2 approches

## 📊 Tableau récapitulatif

| Aspect | Reactive Agent | Planning Agent (Simple)
|--------|---------------|------------------------|
| **Vitesse** | ⚡⚡⚡ Rapide | ⚡⚡ Moyen |
| **Structure** | ❌ Implicite | ✅ Explicite |
| **Traçabilité** | ⚠️ Limitée | ✅ Bonne |
| **Flexibilité** | ✅✅ Très flexible | ⚠️ Rigide |
| **Complexité code** | ⚡ Simple (1 ligne) | ⚡⚡ Moyenne |
| **Fiabilité** | ⚠️ Variable | ✅ Bonne |
| **Debugging** | ❌ Difficile | ✅ Facile |
| **Coût (tokens)** | 💰 Moyen | 💰💰 Élevé (2x LLM) |

### 🎯 Quand utiliser chaque approche?

**Reactive Agent:**
- ✅ Requêtes simples et directes
- ✅ Besoin de réponse rapide
- ✅ Contexte changeant dynamiquement
- ❌ Tâches multi-étapes complexes

**Planning Agent:**
- ✅ Tâches complexes nécessitant décomposition
- ✅ Besoin de traçabilité et audit
- ✅ Workflows répétables
- ✅ Debugging et validation importantes
- ❌ Requêtes très simples (overhead inutile)



## 🎓 Concepts clés appris

### 1. ReAct Pattern (Reactive)
```
Thought → Action → Observation → Thought → ...
```
- Raisonnement entrelacé avec l'action
- Adaptation dynamique
- Boucle continue jusqu'à résolution

### 2. Plan-and-Execute Pattern
```
Plan → Parse → Execute → Synthesize
```
- Décomposition explicite
- Exécution séquentielle
- Meilleure prédictibilité

### 3. Trade-offs
- **Latence vs Structure**: Plus de structure = plus de temps
- **Flexibilité vs Fiabilité**: Réactivité = flexible mais moins prédictible
- **Coût vs Qualité**: Planning = 2x appels LLM mais meilleure qualité

### 4. Agentic AI Design Patterns
Ces deux patterns sont **complémentaires** dans l'écosystème Agentic AI:
- Reactive pour interactions utilisateur
- Planning pour workflows backend
- Hybride possible: plan global + exécution réactive

## 💡 Exercices supplémentaires

Pour approfondir votre compréhension:

1. **Créer un agent hybride** qui décide dynamiquement s'il doit planifier ou réagir

2. **Ajouter de la mémorisation** entre les étapes du planning agent

3. **Implémenter un système de retry** si une sous-tâche échoue

4. **Créer un orchestrateur** qui distribue les sous-tâches à plusieurs agents spécialisés

5. **Ajouter du logging structuré** pour tracer complètement les décisions

6. **Mesurer les performances** (temps, tokens, précision) des deux approches

7. **Créer un visualisateur** du plan d'exécution (graphe de dépendances)

## ✅ Conclusion

Ce lab vous a permis de:

1. ✅ Comprendre les différences fondamentales entre agents réactifs et planificateurs
2. ✅ Implémenter les deux patterns avec LangChain
3. ✅ Comparer leurs performances sur des tâches réelles
4. ✅ Créer un agent planificateur avancé avec parsing et exécution structurée
5. ✅ Identifier les cas d'usage appropriés pour chaque approche

### Points clés à retenir:

- **Reactive agents** = rapides, flexibles, moins structurés
- **Planning agents** = structurés, traçables, plus fiables sur tâches complexes
- Le choix dépend du **contexte**, des **exigences**, et du **budget**
- Les deux patterns sont **complémentaires** dans une architecture Agentic AI

### Ressources pour aller plus loin:

- [LangChain Documentation - Agents](https://python.langchain.com/docs/modules/agents/)
- [ReAct Paper](https://arxiv.org/abs/2210.03629)
- [Plan-and-Solve Paper](https://arxiv.org/abs/2305.04091)
- [LangChain Plan-and-Execute](https://python.langchain.com/docs/use_cases/more/agents/autonomous_agents/plan_and_execute)

---

**🎉 Félicitations! Vous avez complété le LAB31 sur Planning vs Reactive Loops!**